In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import os
import time
from datetime import timedelta
import h5py
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import seaborn as sns

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
fname = h5py.File('/content/drive/MyDrive/archive/greyscale_svhn1.h5', 'r')
X_train = fname['X_train'][:]
y_train = fname['y_train'][:]
X_test = fname['X_test'][:]
y_test = fname['y_test'][:]
X_val = fname['X_val'][:]
y_val = fname['y_val'][:]
fname.close()

In [4]:
print(X_train.shape)

(65931, 32, 32, 1)


In [5]:
print(y_train.shape)

(65931, 10)


In [6]:
#function to plot the img
def plot_img(img, nrows, ncols, cls_true, cls_pred=None):
    fig, axes = plt.subplots(nrows, ncols)
    rs = np.random.choice(img.shape[0], nrows*ncols)
    for i, ax in zip(rs, axes.flat):
        if cls_pred is None:
            title = "True: {0}".format(np.argmax(cls_true[i]))
        else:
            title = "True: {0}, Pred: {1}".format(np.argmax(cls_true[i]), cls_pred[i])
        ax.imshow(img[i,:,:,0], cmap='binary')
        ax.set_title(title)
        ax.set_xticks([])
        ax.set_yticks([])

In [8]:
TENSORBOARD_SUMMARIES_DIR = '/content/drive/MyDrive/archive/svhn_logs' #can be used with pytorch

In [9]:
def log_dir():
  if tf.gfile.Exists(TENSORBOARD_SUMMARIES_DIR):
    tf.gfile.DeleteRecursively(TENSORBOARD_SUMMARIES_DIR)
  tf.gfile.MakeDirs(TENSORBOARD_SUMMARIES_DIR)

In [33]:
def fetch_batch(X, y, batch_size=256):
  for i in np.arange(0,y.shape[0],batch_size):
    last = min(X.shape[0], i+batch_size)
    yield(X[i:last],y[i:last])

In [19]:
comp = 32*32
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
x = tf.placeholder(tf.float32, shape = [None, 32, 32, 1], name='Input_Data')
y = tf.placeholder(tf.float32, shape = [None, 10], name='Input_Labels')
y_cls = tf.argmax(y, 1)
dropout_rate = tf.placeholder(tf.float32, name='Dropout_rate')

In [ ]:
#!tf_upgrade_v2

In [20]:
def svhn_model(X):
  ip = tf.reshape(X, [-1,32,32,1], name = 'reshaped_ip')
  cl1 = tf.layers.conv2d(inputs=ip, filters=32, kernel_size=[5,5], padding='same', activation=tf.nn.relu)
  pool1=tf.layers.max_pooling2d(inputs=cl1,pool_size=[2,2], strides=2)
  cl2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5,5], padding='same', activation=tf.nn.relu)
  pool2=tf.layers.max_pooling2d(inputs=cl2,pool_size=[2,2], strides=2)
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64]) #flattening the op of pool2
  fcnn=tf.layers.dense(inputs=pool2_flat, units=128, activation=tf.nn.relu)
  dropout_layer = tf.layers.dropout(inputs=fcnn, rate=dropout_rate)
  logits = tf.layers.dense(inputs=dropout_layer, units=10)
  return logits

In [21]:
num_ex = X_train.shape[0]
num_epochs = 3

In [22]:
log_dir()

In [23]:
pred = svhn_model(x)
pred_cls = tf.argmax(pred,1)

<ipython-input-20-58e3fb6a768a>:3: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  cl1 = tf.layers.conv2d(inputs=ip, filters=32, kernel_size=[5,5], padding='same', activation=tf.nn.relu)
<ipython-input-20-58e3fb6a768a>:4: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pool1=tf.layers.max_pooling2d(inputs=cl1,pool_size=[2,2], strides=2)
<ipython-input-20-58e3fb6a768a>:5: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  cl2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5,5], padding='same', activation=tf.nn.relu)
<ipython-input-20-58e3fb6a768a>:6: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pool2=tf.layers.ma

In [24]:
loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y,logits=pred))

In [25]:
opt = tf.train.AdamOptimizer().minimize(loss)

In [26]:
pos_pred=tf.equal(pred_cls,y_cls)
acc=tf.reduce_mean(tf.cast(pos_pred,tf.float32))

In [28]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [29]:
saver_tf = tf.train.Saver()
saving_directory = '/content/drive/MyDrive/archive/save_checkpoint'

In [30]:
saving_path = os.path.join(saving_directory, 'svhn1_grey')

In [31]:
batch_size = 512
dropout_precentatge = 0.5

In [34]:
train_loss = []
valid_loss = []
start_time = time.time()
for epoch in range(num_epochs):
    print ('Training .........')
    epoch_loss = 0
    print ()
    print ('Epoch ', epoch+1 , ': ........ \n')
    step = 0

    ## Training epochs ....
    for (epoch_x , epoch_y) in fetch_batch(X_train, y_train, batch_size):
        _, train_accu, c = session.run([opt, acc, loss], feed_dict={x: epoch_x, y: epoch_y, dropout_rate: dropout_precentatge})
        train_loss.append(c)

        if(step%40 == 0):
            print ("Step:", step, ".....", "\nMini-Batch Loss   : ", c)
            print('Mini-Batch Accuracy :' , train_accu*100.0, '%')

            ## Validating prediction and summaries
            accu = 0.0
            for (epoch_x , epoch_y) in fetch_batch(X_val, y_val, 512):
                correct, _c = session.run([pos_pred, loss], feed_dict={x: epoch_x, y: epoch_y, dropout_rate: 0.0})
                valid_loss.append(_c)
                accu+= np.sum(correct[correct == True])
            print('Validation Accuracy :' , accu*100.0/y_val.shape[0], '%')
            print ()
        step = step + 1


    print ('Epoch', epoch+1, 'completed out of ', num_epochs)


## Calculate net time
time_diff = time.time() - start_time

## Testing prediction and summaries
accu = 0.0
for (epoch_x , epoch_y) in fetch_batch(X_test, y_test, 512):
    correct = session.run([pos_pred], feed_dict={x: epoch_x, y: epoch_y, dropout_rate: 0.0})
    accu+= np.sum(correct[correct == True])
print('Test Accuracy :' , accu*100.0/y_test.shape[0], '%')
print("Time usage: " + str(timedelta(seconds=int(round(time_diff)))))
print ()

Training .........

Epoch  1 : ........ 

Step: 0 ..... 
Mini-Batch Loss   :  2.3368883
Mini-Batch Accuracy : 8.0078125 %
Validation Accuracy : 19.51951951951952 %

Step: 40 ..... 
Mini-Batch Loss   :  1.3540759
Mini-Batch Accuracy : 58.3984375 %
Validation Accuracy : 61.42506142506142 %

Step: 80 ..... 
Mini-Batch Loss   :  0.76037306
Mini-Batch Accuracy : 76.7578125 %
Validation Accuracy : 79.78432978432978 %

Step: 120 ..... 
Mini-Batch Loss   :  0.6212231
Mini-Batch Accuracy : 83.0078125 %
Validation Accuracy : 82.30958230958231 %

Epoch 1 completed out of  3
Training .........

Epoch  2 : ........ 

Step: 0 ..... 
Mini-Batch Loss   :  0.62487376
Mini-Batch Accuracy : 81.8359375 %
Validation Accuracy : 84.13868413868414 %

Step: 40 ..... 
Mini-Batch Loss   :  0.5693389
Mini-Batch Accuracy : 85.7421875 %
Validation Accuracy : 84.54818454818455 %

Step: 80 ..... 
Mini-Batch Loss   :  0.53238857
Mini-Batch Accuracy : 84.375 %
Validation Accuracy : 85.40813540813541 %

Step: 120 ..... 

In [35]:
saver_tf.save(sess=session, save_path=saving_path)

'/content/drive/MyDrive/archive/save_checkpoint/svhn1_grey'

In [1]:
#cm, roc-auc, pre-recall curve
#imaug

In [ ]:
#pixel shuffle (super-resolution)